In [1]:
from glob import glob
import pandas as pd
DATA_DIR = "/data/data/fintuning/wanjuan_exam/part-003756-a894b46e.jsonl"
from tqdm import tqdm
import json
import re
import random

In [13]:
def format_example(item):
    example = item["q_main"] + "\n"
    choice_map = {"A":"option_a", "B":"option_b", "C":"option_c", "D":"option_d"} 
    for choice in ["A","B","C","D"]:
        if item[choice_map[choice]] != "":
            example += f'{choice}. {item[choice_map[choice]]}\n'
    # answer = f'{item["std_ans"]}。{item["answer_detail"]}'
    answer = f'{item["answer_detail"]}'
    pattern = r"故选.*"
    answer = re.sub(pattern, '', answer, flags=re.DOTALL)
    answer = answer + f'答案为{item["std_ans"]}'
    return example, answer

all_data = []
with open(DATA_DIR, 'r') as file:
    for idx, line in enumerate(file):
        # 解码JSON对象
        json_obj = json.loads(line)
        if json_obj["std_ans"]=="" or len(json_obj["answer_detail"])<10:
            continue
        # 处理json_obj
        choice, answer = format_example(json_obj)
        # f"以下是一道单选题:\n{question}请给出答案。\n"
        instruct = f"以下是一道{json_obj['q_type']}:\n{choice}请先给出解释再给出答案。\n"
        a_data = {
                "instruction": instruct,
                "input": "",
                "output": answer
            }
        # print(a_data)
        all_data.append(a_data)



In [15]:
print("filter before:", len(all_data))
filted_all_data = [x for x in all_data if "略" not in x["output"]]
print("filter after:", len(all_data))

filter before: 1748506
filter after: 1748506


In [16]:
print(filted_all_data[100])

{'instruction': '以下是一道单选题:\n恰当的安全措施能减少生命财产损失．下列事故处理措施正确的是( )\nA. 油锅着火，立即盖上锅盖\nB. 电器着火，立即用水扑灭\nC. 高层住房着火，立即、打开所有门窗\nD. 煤气泄漏，立即打开排气扇电源开关\n请先给出解释再给出答案。\n', 'input': '', 'output': 'A、油锅着火，立即盖上锅盖，可以隔绝空气达到灭火的目的，故A正确；  \nB、电器着火，立即用水扑灭，会使人触电，造成伤亡事故，故B错误；  \nC、高层住房着火，立即大开所有门窗，因为有了充足的氧气会使火势更猛，故C错误；  \nD、煤气泄漏，立即打开排气扇电源开关，会引起爆炸，故D错误．  \n答案为A'}


In [17]:
import json
json_dir = "/data/data/fintuning/llamafactory_input/wanjuan_exam_270w_cot.jsonl"
with open(json_dir, 'w', encoding="utf-8") as f:
    for item in filted_all_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

In [4]:
def to_alpaca_data(str_json_data):
    return {
                "instruction": str_json_data[-2]["value"],
                "input": "",
                "output": str_json_data[-1]["value"]
            }

def process_choice_data(str_json_data):
    if "选题]" in str_json_data:
        # print(str_json_data)
        return str_json_data
    else:
        return ""

def process_one_data(file_dir):
    table = pq.read_table(file_dir)
    df = table.to_pandas()
    # Index(['id', 'conversations', 'label', 'langdetect', 'source'], dtype='object')
    # print(df.columns)
    # 可以过滤=======================
    # need_idx = df["langdetect"] == "zh-cn"
    # df = df.loc[need_idx, :]
    #===============================
    text_list = df["text"]
    muti_turn_counter = 0
    odd_cov_counter = 0
    a_file_datas = []
    for str_json_data in  text_list:
        res = process_choice_data(str_json_data)
        if res != "":
            a_file_datas.append(res)
    return a_file_datas 

all_datas = []
for file_dir in tqdm(all_files):
    datas = process_one_data(file_dir) 
    all_datas.extend(datas) 
print(len(all_datas))

100%|██████████| 4/4 [00:38<00:00,  9.67s/it]

713702


before data num: 713702
after data num: 76536


In [34]:
import json
print(final_data[0])
json_dir = "/data/data/fintuning/llama3_syne_7w_choice.json"
with open(json_dir, "w") as file:
    json.dump(final_data, file)

{'instruction': '以下是一道选择题，请直接给出答案。下列说法正确的是\n\nA. 分离液态空气获得氮气和氧气属于分解反应 \nB. 禁止在饮用水的水源保护区建化工厂 \nC. 二氧化碳在空气中含量增多会引起温室效应，属于空气污染物 \nD. 氧气可以用作火箭发射的燃料 \n\n答案：', 'input': '', 'output': 'B'}


In [36]:
import json
with open("/data/data/fintuning/llama3_syne_7w_choice.json", "r") as file:
    read_data = json.load(file)
print(read_data[3])

{'instruction': '以下是一道选择题，请直接给出答案。与元素的化学性质关系最密切的是\n\nA. 原子的最外层电子数 \nB. 核外电子数 \nC. 中子数 \nD. 核内质子数 \n\n答案：', 'input': '', 'output': 'A'}
